In [1]:
import sys
sys.path.append('/home/franzihe/Documents/Python/Thesis/')

import pandas as pd
import numpy as np
import netCDF4
import createFolder as cF
import matplotlib.pyplot as plt

In [2]:
station = 'Haukeliseter'

In [3]:
txt_dir = '../../../Data/Retrieval/%s/SWC_hourly/' %(station)

In [4]:
m = ['10', '11', '12', '01', '02', '03']

In [5]:
txt_filename = 'hourly_swc_Haukeli.dat.txt'

In [6]:
wind_pat = 'pulsed'


Haukeli_p = pd.read_fwf(txt_dir + wind_pat + '_' + txt_filename,names = 'S')
Haukeli_p

wind_pat = 'upslope'
Haukeli_u = pd.read_fwf(txt_dir + wind_pat + '_' + txt_filename,names = 'S')

In [7]:
def get_netCDF_variable(f, var_name, var, dim):
    v_0m = f.createVariable(varname=var_name, datatype=var.dtype, dimensions=dim, zlib=True)
    v_0m[:] = var[:]
    return(v_0m)

In [8]:
def separate_SWC_to_days(Date,  Haukeli):
    if Date == 20161031 or Date == 20161231 or Date == 20170131 or Date == 20170331:
        if str(Date).split()[0][4:6] == '12':
                next_Date = 20170101
        else:
            if int(str(Date).split()[0][4:6])+1 < 10:
                    next_Date = int(year+'0'+str(int(str(Date).split()[0][4:6])+1)+'01')
            else:
                    next_Date = int(year+str(int(str(Date).split()[0][4:6])+1)+'01')
    elif Date == 20161130:
            next_Date = int(year+str(int(str(Date).split()[0][4:6])+1)+'01')
    elif Date == 20170228:
            next_Date = 20170301
    else:
            next_Date = Date+1
            
    idx_ = np.where(Haukeli['S'] == Date)[0] 
    idx_2 = np.where(Haukeli['S'] == next_Date)[0]
    
    
    if len(idx_2) == 0:
            _dd = pd.DataFrame(Haukeli['S'][idx_[0]+1 : ]).reset_index()
    else:
            _dd = pd.DataFrame(Haukeli['S'][idx_[0]+1 : idx_2[0]]).reset_index()
        
    if len(_dd) == 0:
            print(Date, 'no data')
            _s = np.empty(shape = (14,60*24))
            _s[:] = np.nan
    else:
            hh = pd.DataFrame()
            _s = np.empty(shape = (14,60*24))
            _s[:] = np.nan
            for i in range(0,24):
                _h = _dd['S'][np.where(_dd['S'] == i*60)[0][0]+1  : np.where(_dd['S'] == i*60)[0][0]+15]
                hh[str(int(_dd['S'][np.where(_dd['S'] == i*60)[0][0]]))] = _h[:].reset_index()['S']
         #   hh = np.array(hh)
            for i in range(0,60*24,60):
                _s[:,i] = hh[str(i)][:]
            
    return _s

In [9]:
for month in m:
    if month == '10':
        t = np.arange(16,32)
    if month == '11':
        t = np.arange(1,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '10' or month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
        
    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day
        
        Date = int(year+month+day)
#        print(Date)
        hh_u = separate_SWC_to_days(Date, Haukeli_u)
        hh_p = separate_SWC_to_days(Date, Haukeli_p)
        
        ### write netCDF file
        cF.createFolder(txt_dir )
        fw = netCDF4.Dataset(txt_dir + '%s.nc' %Date, 'w')
        fw.createDimension('time', hh_u.shape[1])
        fw.createDimension('level', hh_u.shape[0])

        hh_u_w = get_netCDF_variable(fw, 'upslope_SWC', hh_u, ('level', 'time',))
        hh_p_w = get_netCDF_variable(fw, 'pulsed_SWC', hh_p, ('level', 'time',))
        fw.close()

20161216 no data
20161216 no data
20161217 no data
20161217 no data
20161218 no data
20161218 no data
20170130 no data
20170130 no data
20170131 no data
20170131 no data
20170207 no data
20170207 no data
20170208 no data
20170208 no data
20170209 no data
20170209 no data
20170210 no data
20170210 no data
20170211 no data
20170211 no data
20170212 no data
20170212 no data
20170213 no data
20170213 no data
20170214 no data
20170214 no data
20170215 no data
20170215 no data
20170216 no data
20170216 no data
20170217 no data
20170217 no data
20170218 no data
20170218 no data
20170219 no data
20170219 no data
20170220 no data
20170220 no data
20170221 no data
20170221 no data
20170222 no data
20170222 no data
20170223 no data
20170223 no data
20170224 no data
20170224 no data
20170225 no data
20170225 no data
20170226 no data
20170226 no data
20170227 no data
20170227 no data
20170228 no data
20170228 no data
20170301 no data
20170301 no data
20170302 no data
20170302 no data
20170303 no da

In [11]:
hh_u.shape

(14, 1440)